In [1]:
import numpy as np
from dataclasses import dataclass
from typing import Callable, Dict, Optional, Tuple
import matplotlib.pyplot as plt
from math import comb

# --------------------------------------------------------------------
# Simplex discretization for base-stock policies
# --------------------------------------------------------------------
def simplex_grid(n: int, m: int) -> np.ndarray:
    """
    Construct a uniform grid on the (n-1)-simplex:
        { z in R^n : z_i >= 0, sum_i z_i = 1 }
    with coordinates in {0, 1/m, 2/m, ..., 1}.

    Parameters
    ----------
    n : int
        Number of locations (dimension of the simplex).
    m : int
        Resolution parameter; larger m gives finer grid and more arms.

    Returns
    -------
    grid : np.ndarray, shape (K, n)
        Each row is a base-stock vector b in Δ_{n-1}.
    """
    if n < 2:
        raise ValueError("n must be at least 2.")
    if m < 1:
        raise ValueError("m must be at least 1.")

    grid = []

    def _rec(prefix, remaining, dim):
        # Recursive construction of integer compositions of 'm' into 'n' parts.
        if dim == n - 1:
            # Last coordinate determined by what's left.
            prefix.append(remaining)
            grid.append(np.array(prefix, dtype=float) / m)
            prefix.pop()
            return

        for k in range(remaining + 1):
            prefix.append(k)
            _rec(prefix, remaining - k, dim + 1)
            prefix.pop()

    _rec([], m, 0)
    return np.vstack(grid)


# --------------------------------------------------------------------
# Network inventory environment
# --------------------------------------------------------------------
class NetworkInventoryEnv:
    """
    Closed network inventory MDP with n locations.

    - State x_t is a probability vector in the simplex Δ_{n-1}.
    - Action y_t is a base-stock vector in the simplex Δ_{n-1}.
    - Dynamics: x_{t+1} = (y_t - d_t)^+ + P_t^T min(y_t, d_t),
      followed by normalization to stay on the simplex.
    - Repositioning cost: M(y_t - x_t) with uniform per-unit cost c_ij ≡ c.
    - Lost-sales cost: l * sum_i (d_{t,i} - y_{t,i})^+.
    - Modified cost: \tilde{C}_t = M(y_t - x_t) - l * sum_i min(d_{t,i}, y_{t,i}).
    """

    def __init__(
        self,
        n: int,
        demand_sampler: Callable[[np.random.Generator, int], np.ndarray],
        routing_sampler: Callable[[np.random.Generator, int], np.ndarray],
        cost_reposition: float = 1.0,
        cost_lost: float = 5.0,
        x0: Optional[np.ndarray] = None,
        rng: Optional[np.random.Generator] = None,
    ):
        """
        Parameters
        ----------
        n : int
            Number of locations.
        demand_sampler : callable
            Function (rng, n) -> demand vector d in R_+^n.
        routing_sampler : callable
            Function (rng, n) -> row-stochastic routing matrix P in R^{n x n}.
        cost_reposition : float
            Uniform per-unit repositioning cost c.
        cost_lost : float
            Uniform lost-sales cost per unit l.
        x0 : np.ndarray, optional
            Initial state; if None, use uniform distribution.
        rng : np.random.Generator, optional
            Random number generator.
        """
        self.n = n
        self.demand_sampler = demand_sampler
        self.routing_sampler = routing_sampler
        self.cost_reposition = float(cost_reposition)
        self.cost_lost = float(cost_lost)
        self.rng = rng or np.random.default_rng()

        if x0 is None:
            self.x = np.ones(n, dtype=float) / n
        else:
            x0 = np.asarray(x0, dtype=float)
            if x0.shape != (n,):
                raise ValueError("x0 must have shape (n,)")
            if x0.sum() <= 0:
                raise ValueError("x0 must have positive total inventory.")
            self.x = x0 / x0.sum()

    @property
    def state(self) -> np.ndarray:
        """Current inventory distribution x_t."""
        return self.x

    def reset(self, x0: Optional[np.ndarray] = None) -> np.ndarray:
        """
        Reset environment to initial state.

        Parameters
        ----------
        x0 : np.ndarray, optional
            Initial state; if None, uses uniform.

        Returns
        -------
        x : np.ndarray
            Reset state.
        """
        if x0 is None:
            self.x = np.ones(self.n, dtype=float) / self.n
        else:
            x0 = np.asarray(x0, dtype=float)
            if x0.shape != (self.n,):
                raise ValueError("x0 must have shape (n,)")
            if x0.sum() <= 0:
                raise ValueError("x0 must have positive total inventory.")
            self.x = x0 / x0.sum()
        return self.x

    def reposition_cost(self, x_from: np.ndarray, y_to: np.ndarray) -> float:
        """
        Minimal repositioning cost M(y - x) under uniform transportation cost.

        We specialize to c_ij = c (constant) for i != j and c_ii = 0.
        In this case the optimal cost is

            M(y - x) = c * 0.5 * ||y - x||_1,

        because the total mass moved equals half the L1 distance.
        """
        x_from = np.asarray(x_from, dtype=float)
        y_to = np.asarray(y_to, dtype=float)
        if x_from.shape != (self.n,) or y_to.shape != (self.n,):
            raise ValueError("x_from and y_to must have shape (n,)")

        return 0.5 * self.cost_reposition * np.abs(y_to - x_from).sum()

    def step(
        self, y: np.ndarray
    ) -> Tuple[np.ndarray, float, float, float, Dict[str, np.ndarray]]:
        """
        Apply target inventory level y, realize demand and routing, and update state.

        Parameters
        ----------
        y : np.ndarray
            Target inventory distribution in the simplex Δ_{n-1}.

        Returns
        -------
        x_next : np.ndarray
            Next state x_{t+1}.
        total_cost : float
            Single-period cost C_t = M(y_t - x_t) + lost-sales cost.
        modified_cost : float
            Observable modified cost \tilde{C}_t.
        repo_cost : float
            Repositioning cost M(y_t - x_t).
        info : dict
            Dictionary with keys 'demand', 'routing', 'sold', 'lost'.
        """
        y = np.asarray(y, dtype=float)
        if y.shape != (self.n,):
            raise ValueError("y must have shape (n,)")
        if np.any(y < -1e-8):
            raise ValueError("y must be nonnegative.")
        total_y = y.sum()
        if not np.isclose(total_y, 1.0):
            # Normalize to stay on the simplex if user passes an unnormalized vector.
            y = y / total_y

        x_prev = self.x.copy()
        # Repositioning cost from current state to target base-stock vector.
        repo_cost = self.reposition_cost(x_prev, y)

        # Realize demand and routing.
        d = np.asarray(self.demand_sampler(self.rng, self.n), dtype=float)
        if d.shape != (self.n,):
            raise ValueError("demand_sampler must return shape (n,)")
        if np.any(d < 0):
            raise ValueError("Demand must be nonnegative.")

        P = np.asarray(self.routing_sampler(self.rng, self.n), dtype=float)
        if P.shape != (self.n, self.n):
            raise ValueError("routing_sampler must return shape (n, n)")

        # Ensure each row of P is a probability distribution.
        row_sums = P.sum(axis=1, keepdims=True)
        P = np.divide(
            P,
            row_sums,
            out=np.full_like(P, 1.0 / self.n),
            where=row_sums > 0,
        )

        # Censored sales and lost demand.
        sold = np.minimum(y, d)           # fulfilled demand min(y_i, d_i)
        lost = np.maximum(d - y, 0.0)     # lost demand (d_i - y_i)^+

        # Inventory dynamics: (y - d)^+ + P^T * sold.
        leftover = y - sold
        x_next = leftover + P.T @ sold

        # Numerical tidy-up: renormalize to stay on the simplex.
        total = x_next.sum()
        if total > 0:
            x_next /= total

        # Lost-sales cost (uniform loss cost per unit).
        lost_sales_cost = self.cost_lost * lost.sum()

        total_cost = repo_cost + lost_sales_cost

        # Modified cost: M(y - x_t) - l * sum_i min(y_i, d_i)
        modified_cost = repo_cost - self.cost_lost * sold.sum()

        # Commit state transition.
        self.x = x_next

        info = {
            "demand": d,
            "routing": P,
            "sold": sold,
            "lost": lost,
        }

        return x_next, total_cost, modified_cost, repo_cost, info


# --------------------------------------------------------------------
# LipBR configuration and algorithm
# --------------------------------------------------------------------
@dataclass
class LipBRConfig:
    horizon: int
    grid_m: int = 4      # resolution of simplex discretization
    H: float = 5.0       # UCB exploration parameter


class LipBR:
    """
    Lipschitz Bandits-based Repositioning (LipBR) algorithm.

    - Discretizes the base-stock policy space by a grid on Δ_{n-1}.
    - Treats each grid point as an arm.
    - Uses an epoch-based UCB strategy on pseudo modified costs with
      memory points to approximate the "consecutive play" requirement
      in the concentration inequalities.
    """

    def __init__(self, env: NetworkInventoryEnv, config: LipBRConfig):
        self.env = env
        self.cfg = config
        self.n = env.n

        # Discretize base-stock policies (arms).
        self.B = simplex_grid(self.n, self.cfg.grid_m)  # shape (K, n)
        self.K = self.B.shape[0]

        # Epoch length per arm (N_k in the paper), start with 1.
        self.epoch_lengths = np.ones(self.K, dtype=int)

        # Number of times each arm has been played (τ_k).
        self.tau = np.zeros(self.K, dtype=int)

        # Sum of pseudo modified costs per arm (for empirical means).
        self.sum_pseudo_cost = np.zeros(self.K, dtype=float)

        # UCB values per arm (initialize to +∞ to force exploration).
        self.UCB = np.full(self.K, np.inf, dtype=float)

        # Memory points m_k: last state at end of an epoch where arm k was used.
        self.memory_state = np.tile(self.env.state, (self.K, 1))

    def run(self, verbose: bool = False) -> Dict[str, np.ndarray]:
        """
        Run LipBR for the given time horizon.

        Parameters
        ----------
        verbose : bool
            If True, prints basic epoch information.

        Returns
        -------
        history : dict
            Dictionary containing per-period statistics and arm-level summaries.
        """
        T = self.cfg.horizon
        H = self.cfg.H

        # Per-period logs.
        chosen_arms = np.empty(T, dtype=int)
        total_costs = np.empty(T, dtype=float)
        modified_costs = np.empty(T, dtype=float)
        pseudo_costs = np.empty(T, dtype=float)
        avg_modified_cost = np.empty(T, dtype=float)

        t = 0
        epoch = 0
        last_arm: Optional[int] = None
        cum_modified_cost = 0.0

        while t < T:
            epoch += 1

            # Select arm with highest UCB (ties broken by smallest index).
            k = int(np.argmax(self.UCB))
            base_stock = self.B[k]
            N_k = self.epoch_lengths[k]

            if verbose:
                print(
                    f"Epoch {epoch}: arm {k}, length {N_k}, "
                    f"tau={self.tau[k]}, UCB={self.UCB[k]:.3f}"
                )

            # Play arm k for N_k consecutive periods (or until horizon ends).
            for _ in range(N_k):
                if t >= T:
                    break

                x_prev = self.env.state.copy()

                (x_next,
                 total_cost,
                 mod_cost,
                 repo_cost_prev,
                 info) = self.env.step(base_stock)

                # Pseudo modified cost adjustment (equation analogue (21)):
                #   \tilde{C}'_t = \tilde{C}_t + M(b_k - m_k) - M(b_k - x_t)
                # when we switch into arm k.
                if last_arm is None or k == last_arm:
                    pseudo_cost = mod_cost
                else:
                    repo_cost_mem = self.env.reposition_cost(
                        self.memory_state[k], base_stock
                    )
                    pseudo_cost = mod_cost + repo_cost_mem - repo_cost_prev

                # Update statistics for arm k.
                self.sum_pseudo_cost[k] += pseudo_cost
                self.tau[k] += 1

                # Logging per-period data.
                chosen_arms[t] = k
                total_costs[t] = total_cost
                modified_costs[t] = mod_cost
                pseudo_costs[t] = pseudo_cost

                cum_modified_cost += mod_cost
                avg_modified_cost[t] = cum_modified_cost / (t + 1)

                t += 1
                last_arm = k

            # End of epoch for arm k: update memory point and its UCB.
            self.memory_state[k] = self.env.state.copy()

            # Empirical mean pseudo cost for arm k.
            mean_pseudo_cost = self.sum_pseudo_cost[k] / max(self.tau[k], 1)
            # Treat "reward" as negative cost and apply UCB.
            self.UCB[k] = -mean_pseudo_cost + H * np.sqrt(
                np.log(T) / max(self.tau[k], 1)
            )

            # Double the epoch length for this arm.
            self.epoch_lengths[k] *= 2

        # Summarize per-arm performance.
        mean_pseudo_per_arm = np.full(self.K, np.nan, dtype=float)
        for k in range(self.K):
            if self.tau[k] > 0:
                mean_pseudo_per_arm[k] = self.sum_pseudo_cost[k] / self.tau[k]

        # Approximate pseudo-regret using in-sample best arm in discretized set.
        valid = np.isfinite(mean_pseudo_per_arm)
        if valid.any():
            best_mean = np.nanmin(mean_pseudo_per_arm[valid])
            cum_pseudo = np.cumsum(pseudo_costs)
            approx_pseudoregret = cum_pseudo - best_mean * np.arange(1, T + 1)
        else:
            approx_pseudoregret = np.full(T, np.nan)

        # Pseudo-regret vs last chosen arm (as benchmark)
        if T > 0:
            last_arm_final = int(chosen_arms[T - 1])
        else:
            last_arm_final = None
        if last_arm_final is not None and self.tau[last_arm_final] > 0 and np.isfinite(mean_pseudo_per_arm[last_arm_final]):
            mu_last = float(mean_pseudo_per_arm[last_arm_final])
            last_arm_pseudoregret = np.cumsum(pseudo_costs) - mu_last * np.arange(1, T + 1)
            last_arm_mean = mu_last
        else:
            last_arm_pseudoregret = np.full(T, np.nan)
            last_arm_mean = np.nan

        history = {
            "chosen_arms": chosen_arms,
            "total_cost": total_costs,
            "modified_cost": modified_costs,
            "pseudo_cost": pseudo_costs,
            "avg_modified_cost": avg_modified_cost,
            "mean_pseudo_cost_per_arm": mean_pseudo_per_arm,
            "base_stocks": self.B,
            "approx_pseudoregret": approx_pseudoregret,
            "last_arm": last_arm_final if T > 0 else None,
            "last_arm_mean_pseudo_cost": last_arm_mean,
            "last_arm_pseudoregret": last_arm_pseudoregret,
        }
        return history



def recommended_grid_m(
    T: int,
    n: int,
    cost_reposition: float = 1.0,
    cost_lost: float = 5.0,
    safety_factor: float = 1.0,
    max_arms: int = 5000,
) -> int:
    """
    Recommend a grid resolution m for simplex_grid(n, m) that roughly
    matches the theoretical scaling δ(T) ~ (log T / T)^{1/(n+1)}.

    Theory sketch:
        - Discretization radius δ ≍ (log T / T)^{1/(n+1)}
        - Our grid step is 1/m ⇒ δ ≍ 1/m ⇒ m ≍ (T / log T)^{1/(n+1)}.

    We also cap the number of arms K = C(m + n - 1, n - 1) by 'max_arms'
    to keep the numerical illustration fast and readable.

    Parameters
    ----------
    T : int
        Time horizon.
    n : int
        Number of locations (dimension of the simplex).
    cost_reposition : float
        Reposition cost parameter c; used to estimate Lipschitz constant.
    cost_lost : float
        Lost-sales cost parameter l; used to estimate Lipschitz constant.
    safety_factor : float
        Multiplicative factor on δ; >1 means coarser grid (smaller m),
        <1 means finer grid (larger m).
    max_arms : int
        Maximum allowed number of arms K = comb(m + n - 1, n - 1).

    Returns
    -------
    m : int
        Suggested grid_m for simplex_grid(n, m).
    """
    if T <= 1:
        return 1

    # Lipschitz constant η ≲ l + 6c (from Lemma in the paper),
    # used only to normalize; we largely care about the T, n dependence.
    eta = cost_lost + 6.0 * cost_reposition

    # Theoretical δ(T) ~ (log T / T)^{1/(n+1)} up to constants;
    # we allow a safety_factor to coarsen/fine-tune in practice.
    delta_raw = (np.log(T) / T) ** (1.0 / (n + 1))

    # Incorporate η and safety_factor as mild rescaling; small δ => larger m.
    # Since η only multiplies the discretization term, we keep this simple.
    delta = safety_factor * delta_raw

    # Translate δ ≈ 1/m  ⇒  m ≈ 1/δ
    m = int(np.ceil(1.0 / delta))
    m = max(1, m)

    # Cap m so that number of arms K does not explode in small experiments.
    # K = (m + n - 1 choose n - 1).
    while m > 1 and comb(m + n - 1, n - 1) > max_arms:
        m -= 1

    return m*100


In [2]:


def simulate_no_reposition(
    env: NetworkInventoryEnv,
    horizon: int,
) -> Dict[str, np.ndarray]:
    """
    Simulate the 'no repositioning' policy:
        y_t = x_t  for all t,
    i.e., never move inventory proactively.

    Parameters
    ----------
    env : NetworkInventoryEnv
        Environment instance (will be mutated).
    horizon : int
        Number of periods to simulate.

    Returns
    -------
    history : dict
        Same structure as the LipBR history for easy comparison:
        - "total_cost"
        - "modified_cost"
        - "avg_modified_cost"
    """
    n = env.n
    T = horizon

    total_costs = np.empty(T, dtype=float)
    modified_costs = np.empty(T, dtype=float)
    avg_modified_cost = np.empty(T, dtype=float)

    # Reset environment to a fresh initial state (optional: pass x0).
    x = env.reset()
    cum_mod = 0.0

    for t in range(T):
        # No repositioning: target equals current inventory.
        y = x.copy()
        y_sum = y.sum()
        if not np.isclose(y_sum, 1.0):
            y = y / y_sum  # ensure on simplex

        x_next, total_cost, mod_cost, repo_cost, info = env.step(y)

        total_costs[t] = total_cost
        modified_costs[t] = mod_cost

        cum_mod += mod_cost
        avg_modified_cost[t] = cum_mod / (t + 1)

        x = x_next

    history = {
        "total_cost": total_costs,
        "modified_cost": modified_costs,
        "avg_modified_cost": avg_modified_cost,
    }
    return history

import numpy as np
from typing import Dict

# --------------------------------------------------------------------
# Uniform repositioning baseline simulation
# --------------------------------------------------------------------
def simulate_uniform_reposition(
    env: NetworkInventoryEnv,
    horizon: int,
) -> Dict[str, np.ndarray]:
    """
    Simulate the 'uniform repositioning' baseline:
        y_t = (1/n, ..., 1/n) for all t.

    This corresponds to a fixed base-stock policy at the center of the simplex.
    At each period, the platform rebalances inventory to the uniform vector,
    regardless of the current state x_t.

    Parameters
    ----------
    env : NetworkInventoryEnv
        Environment instance (will be mutated).
    horizon : int
        Number of periods to simulate.

    Returns
    -------
    history : dict
        Dictionary with:
        - "total_cost": per-period total costs C_t
        - "modified_cost": per-period modified costs \\tilde C_t
        - "avg_modified_cost": running average of \\tilde C_t
    """
    n = env.n
    T = horizon

    # Fixed uniform base-stock vector
    y_uniform = np.ones(n, dtype=float) / n

    total_costs = np.empty(T, dtype=float)
    modified_costs = np.empty(T, dtype=float)
    avg_modified_cost = np.empty(T, dtype=float)

    # Reset environment to a fresh initial state (optional: pass x0 if desired)
    x = env.reset()
    cum_mod = 0.0

    for t in range(T):
        # Use the same base-stock target every period
        x_next, total_cost, mod_cost, repo_cost, info = env.step(y_uniform)

        total_costs[t] = total_cost
        modified_costs[t] = mod_cost

        cum_mod += mod_cost
        avg_modified_cost[t] = cum_mod / (t + 1)

        x = x_next

    history = {
        "total_cost": total_costs,
        "modified_cost": modified_costs,
        "avg_modified_cost": avg_modified_cost,
    }
    return history


In [3]:
import os
import pandas as pd
from datetime import datetime


def save_history(history: Dict[str, np.ndarray], label: str, outdir: str = "numerical_results", extra_meta: Dict[str, str] | None = None, write_metadata_md: bool = False, metadata_md_params: Dict[str, str] | None = None) -> str:
    """
    Save a history dict into a timestamped subfolder for convenient plotting.

    Folder structure:
      numerical_results/<timestamp>/
        - <policy>.csv (policy-specific filename from label)
        - params.md (optional, once per run)

    Per-period arrays go into the CSV. Non-period arrays are skipped.
    """
    now = datetime.now().strftime("%Y%m%d-%H%M%S")
    subdir = os.path.join(outdir, now)
    os.makedirs(subdir, exist_ok=True)

    # Determine horizon T from 1D arrays
    horizon = None
    for v in history.values():
        arr = np.asarray(v)
        if arr.ndim == 1:
            horizon = len(arr) if horizon is None else min(horizon, len(arr))
    if horizon is None:
        horizon = 0

    # Build per-period dataframe
    df_period = pd.DataFrame(index=range(horizon))
    for k, v in history.items():
        arr = np.asarray(v)
        if arr.ndim == 1 and len(arr) >= horizon:
            df_period[k] = arr[:horizon]
        elif arr.ndim == 2 and arr.shape[0] >= horizon:
            for j in range(arr.shape[1]):
                df_period[f"{k}_{j}"] = arr[:horizon, j]
        # Non-period arrays are ignored for CSV

    # Metadata columns embedded in CSV for filtering/grouping
    df_period["run_timestamp"] = now
    df_period["run_label"] = label
    if extra_meta:
        for mk, mv in extra_meta.items():
            df_period[str(mk)] = str(mv)

    # Policy-specific filename: sanitize label
    slug = "".join(ch.lower() if ch.isalnum() else "_" for ch in label).strip("_") or "history"
    fname = f"{slug}.csv"
    period_path = os.path.join(subdir, fname)
    df_period.to_csv(period_path, index=False)

    # Optionally write a single markdown file with parameters (once per run)
    if write_metadata_md:
        md_path = os.path.join(subdir, "params.md")
        lines = ["# Experiment Parameters\n\n"]
        if metadata_md_params:
            # Make a simple list of key-value pairs
            for k, v in metadata_md_params.items():
                lines.append(f"- **{k}**: {v}\n")
        # Also include timestamp and label
        lines.append(f"\n- **timestamp**: {now}\n")
        lines.append(f"- **labels saved**: {label}\n")
        with open(md_path, "a", encoding="utf-8") as f:
            f.writelines(lines)

    return os.path.abspath(period_path)

In [4]:


def multi_horizon_policy_table_markdown(
    histories_by_policy: Dict[str, Dict[int, Dict[str, np.ndarray]]],
    decimals: int = 3,
    title: str | None = "Average total cost across horizons",
) -> str:
    """
    Build a Markdown table where each row is a policy and each column is a
    time horizon T, with entries equal to the average total cost over that
    horizon for the policy.

    Parameters
    ----------
    histories_by_policy : dict
        Mapping from policy_name -> { T -> history_dict }.
        Each history_dict should contain key "total_cost": shape (T,).
        If a policy lacks a particular T, the cell is left blank.
    decimals : int
        Number of decimals to display.
    title : str | None
        Optional title displayed above the table. Set None to omit.

    Returns
    -------
    table_str : str
        Markdown-formatted table (with optional title line),
        rows = policies, columns = distinct horizons.
    """
    if not histories_by_policy:
        return "No histories provided."

    # Collect the set of all horizons appearing across policies.
    all_Ts = set()
    for policy, byT in histories_by_policy.items():
        for T in byT.keys():
            all_Ts.add(int(T))
    sorted_Ts = sorted(all_Ts)

    # Build header row.
    header = "| Policy | " + " | ".join(f"T={T}" for T in sorted_Ts) + " |\n"
    align = "|:-------|" + "|".join([":----------:" for _ in sorted_Ts]) + "|\n"

    # Build each policy row.
    rows = []
    for policy, byT in histories_by_policy.items():
        cells = []
        for T in sorted_Ts:
            hist = byT.get(T)
            if hist is None or "total_cost" not in hist:
                cells.append("")
            else:
                arr = np.asarray(hist["total_cost"])[:T]
                val = arr.mean() if arr.size > 0 else float("nan")
                cells.append(f"{val:.{decimals}f}")
        row = "| " + policy + " | " + " | ".join(cells) + " |"
        rows.append(row)

    # Optional title line
    prefix = f"**{title}**\n\n" if title else ""
    return prefix + header + align + "\n".join(rows)


def lipbr_last_arm_regret_table_markdown(
    histories_by_policy: Dict[str, Dict[int, Dict[str, np.ndarray]]],
    decimals: int = 1,
    title: str | None = "LipBR pseudo-regret vs last arm (final cumulative)",
) -> str:
    """
    Build a one-row Markdown table reporting, for each horizon T, the final
    cumulative pseudo-regret of LipBR relative to its last chosen arm.

    Expects LipBR histories to include keys:
        - "last_arm_pseudoregret" (shape (T,))
    """
    if "LipBR" not in histories_by_policy or not histories_by_policy["LipBR"]:
        return "No LipBR histories provided."

    sorted_Ts = sorted(int(T) for T in histories_by_policy["LipBR"].keys())
    header = "| Metric | " + " | ".join(f"T={T}" for T in sorted_Ts) + " |\n"
    align = "|:-------|" + "|".join([":----------:" for _ in sorted_Ts]) + "|\n"

    cells = []
    for T in sorted_Ts:
        hist = histories_by_policy["LipBR"].get(T)
        if hist is None or "last_arm_pseudoregret" not in hist:
            cells.append("")
        else:
            arr = np.asarray(hist["last_arm_pseudoregret"])[:T]
            val = arr[-1] if arr.size > 0 else float("nan")
            cells.append(f"{val:.{decimals}f}")

    row = "| Final cum. pseudo-regret | " + " | ".join(cells) + " |"
    prefix = f"**{title}**\n\n" if title else ""
    return prefix + header + align + row


def avg_regret_vs_lipbr_last_arm_table_markdown(
    histories_by_policy: Dict[str, Dict[int, Dict[str, np.ndarray]]],
    decimals: int = 3,
    title: str | None = "Average pseudo-regret vs LipBR last arm",
) -> str:
    """
    For each horizon T, compute average pseudo-regret (1/T sum_t (pseudo_t - mu_last))
    for all policies using LipBR's last-arm empirical mean pseudo cost mu_last(T)
    as the benchmark.

    - For LipBR, use history["pseudo_cost"].
    - For baselines (NoRepo, Uniform), use history["modified_cost"] as pseudo costs.
    """
    if "LipBR" not in histories_by_policy or not histories_by_policy["LipBR"]:
        return "No LipBR histories provided."

    # Collect horizons present in LipBR (benchmark availability).
    Ts = sorted(int(T) for T in histories_by_policy["LipBR"].keys())

    # Header
    header = "| Policy | " + " | ".join(f"T={T}" for T in Ts) + " |\n"
    align = "|:-------|" + "|".join([":----------:" for _ in Ts]) + "|\n"

    # Policies to include in rows (intersection with available histories)
    policies = [p for p in ["LipBR", "NoRepo", "Uniform"] if p in histories_by_policy]

    rows = []
    for policy in policies:
        cells = []
        for T in Ts:
            lip_hist = histories_by_policy["LipBR"].get(T)
            pol_hist = histories_by_policy[policy].get(T)
            if lip_hist is None or pol_hist is None:
                cells.append("")
                continue
            mu_last = lip_hist.get("last_arm_mean_pseudo_cost")
            if mu_last is None or not np.isfinite(mu_last):
                cells.append("")
                continue
            # Policy pseudo-cost series
            if policy == "LipBR":
                pc = np.asarray(pol_hist.get("pseudo_cost"))
            else:
                pc = np.asarray(pol_hist.get("modified_cost"))
            if pc is None or pc.size == 0:
                cells.append("")
                continue
            pc = pc[:T]
            avg_regret = (pc.sum() - float(mu_last) * len(pc)) / max(len(pc), 1)
            cells.append(f"{avg_regret:.{decimals}f}")
        row = "| " + policy + " | " + " | ".join(cells) + " |"
        rows.append(row)

    prefix = f"**{title}**\n\n" if title else ""
    return prefix + header + align + "\n".join(rows)


In [ ]:
if __name__ == "__main__":
    # Example: closed inventory network with n locations.
    # You can now provide a list of n values to sweep over.
    n_list = [2, 3, 4]  # e.g., [2, 3, 4]
    T_list = [1000, 2000, 3000]  # multiple horizons for the table
    num_runs = 20  # repeat experiments and report averages
    
    # ------------------------------------------------------------------
    # Automatic demand mean vector generation
    # ------------------------------------------------------------------
    def build_demand_mu(n: int, low: float = 0.2, high: float = 0.8) -> np.ndarray:
        """Create a length-n demand mean vector spanning [low, high].
        If n == 1 returns array([low]).
        """
        if n < 1:
            raise ValueError("n must be >= 1")
        if n == 1:
            return np.array([low], dtype=float)
        return np.linspace(low, high, n, dtype=float)
    
    # ------------------------------------------------------------------
    # Shared-randomness utilities (common random numbers across policies)
    # ------------------------------------------------------------------
    def sample_sequences(n: int, T: int, demand_mu: np.ndarray, seed: int):
        """Pre-sample demand vectors and routing matrices for T periods.
        - demand_seq: shape (T, n)
        - routing_seq: shape (T, n, n), each row is Dirichlet(1,...,1)
        """
        rng_local = np.random.default_rng(seed)
        demand_seq = rng_local.poisson(lam=demand_mu, size=(T, n)).astype(float)
        routing_seq = np.empty((T, n, n), dtype=float)
        for t in range(T):
            routing_seq[t] = rng_local.dirichlet(alpha=np.ones(n), size=n)
        return demand_seq, routing_seq
    
    def make_fixed_samplers(demand_seq: np.ndarray, routing_seq: np.ndarray):
        """Build deterministic samplers that replay the same sequences per period.
        We advance the internal time index once per period (on routing call).
        """
        T, n = demand_seq.shape
        idx = {"t": 0}
        
        def demand_sampler_fixed(rng_: np.random.Generator, n_: int) -> np.ndarray:
            if n_ != n:
                raise ValueError("Mismatch: n_ does not equal pre-sampled n.")
            t = idx["t"]
            if t >= T:
                # Gracefully repeat last value if called beyond horizon
                return demand_seq[-1]
            return demand_seq[t]
        
        def routing_sampler_fixed(rng_: np.random.Generator, n_: int) -> np.ndarray:
            if n_ != n:
                raise ValueError("Mismatch: n_ does not equal pre-sampled n.")
            t = idx["t"]
            if t >= T:
                P = routing_seq[-1]
            else:
                P = routing_seq[t]
            idx["t"] = t + 1
            return P
        
        return demand_sampler_fixed, routing_sampler_fixed
    
    # Cost parameters (kept consistent across policies)
    COST_REPO = 1.0
    COST_LOST = 10.0
    
    # Sweep over the list of n values
    for n in n_list:
        demand_mu = build_demand_mu(n)  # automatically adapts to n
        
        print(f"\n===== Running experiments for n={n} =====")
        
        # Run policies for each T and collect averaged histories across runs.
        histories_by_policy_avg: Dict[str, Dict[int, Dict[str, np.ndarray]]] = {
            "LipBR": {},
            "NoRepo": {},
            "Uniform": {},
        }
        
        for T in T_list:
            # Use recommended grid resolution with a cap on number of arms
            grid_m_T = recommended_grid_m(
                T=T,
                n=n,
                cost_reposition=COST_REPO,
                cost_lost=COST_LOST,
                safety_factor=1.0,
                max_arms=5000,
            )
            
            # Collect per-run histories for averaging
            runs_by_policy: Dict[str, list[Dict[str, np.ndarray]]] = {
                "LipBR": [],
                "NoRepo": [],
                "Uniform": [],
            }
            
            base_seed = 20251128  # base seed for reproducibility across days
            
            for run_idx in range(num_runs):
                # Pre-sample identical sequences for all policies in this run
                run_seed = base_seed + 1000 * T + run_idx  # depends on T and run
                d_seq, P_seq = sample_sequences(n=n, T=T, demand_mu=demand_mu, seed=run_seed)
                
                # Build fixed samplers; each env gets its own index but same sequences
                d_samp_L, P_samp_L = make_fixed_samplers(d_seq, P_seq)
                d_samp_N, P_samp_N = make_fixed_samplers(d_seq, P_seq)
                d_samp_U, P_samp_U = make_fixed_samplers(d_seq, P_seq)
                
                # LipBR for horizon T (shared randomness)
                cfg = LipBRConfig(horizon=T, grid_m=grid_m_T, H=5.0)
                env_lipbr_T = NetworkInventoryEnv(
                    n=n,
                    demand_sampler=d_samp_L,
                    routing_sampler=P_samp_L,
                    cost_reposition=COST_REPO,
                    cost_lost=COST_LOST,
                    rng=np.random.default_rng(123 + run_idx),
                )
                agent = LipBR(env_lipbr_T, cfg)
                runs_by_policy["LipBR"].append(agent.run(verbose=False))
                
                # No repositioning baseline (shared randomness)
                env_norepo_T = NetworkInventoryEnv(
                    n=n,
                    demand_sampler=d_samp_N,
                    routing_sampler=P_samp_N,
                    cost_reposition=COST_REPO,
                    cost_lost=COST_LOST,
                    rng=np.random.default_rng(456 + run_idx),
                )
                runs_by_policy["NoRepo"].append(
                    simulate_no_reposition(env_norepo_T, horizon=T)
                )
                
                # Uniform repositioning baseline (shared randomness)
                env_uniform_T = NetworkInventoryEnv(
                    n=n,
                    demand_sampler=d_samp_U,
                    routing_sampler=P_samp_U,
                    cost_reposition=COST_REPO,
                    cost_lost=COST_LOST,
                    rng=np.random.default_rng(789 + run_idx),
                )
                runs_by_policy["Uniform"].append(
                    simulate_uniform_reposition(env_uniform_T, horizon=T)
                )
            
            # --------------------------------------------------------------
            # Average metrics across runs for each policy
            # --------------------------------------------------------------
            def average_histories(hist_list: list[Dict[str, np.ndarray]], T: int) -> Dict[str, np.ndarray]:
                if not hist_list:
                    return {}
                out: Dict[str, np.ndarray] = {}
                
                # 1D time series keys we care about
                keys_1d = [
                    "total_cost",
                    "modified_cost",
                    "avg_modified_cost",
                    "pseudo_cost",
                    "approx_pseudoregret",
                    "last_arm_pseudoregret",
                ]
                for k in keys_1d:
                    series = []
                    for h in hist_list:
                        if k in h and np.asarray(h[k]).ndim == 1:
                            arr = np.asarray(h[k])[:T]
                            if arr.size == T:
                                series.append(arr)
                    if series:
                        out[k] = np.mean(np.stack(series, axis=0), axis=0)
                
                # Scalar keys (averages across runs)
                scalar_keys = ["last_arm_mean_pseudo_cost"]
                for k in scalar_keys:
                    vals = []
                    for h in hist_list:
                        if k in h:
                            v = h[k]
                            if np.isscalar(v) or (isinstance(v, np.ndarray) and np.asarray(v).ndim == 0):
                                try:
                                    vals.append(float(v))
                                except Exception:
                                    pass
                    if vals:
                        out[k] = float(np.mean(vals))
                
                return out
            
            for policy in ["LipBR", "NoRepo", "Uniform"]:
                histories_by_policy_avg[policy][T] = average_histories(
                    runs_by_policy[policy], T
                )
        
        # Print matrix-style Markdown table: policies x horizons (avg total cost), averaged over runs.
        print(
            f"\n[ n={n} ] " +
            multi_horizon_policy_table_markdown(
                histories_by_policy_avg, decimals=3,
                title=f"Average total cost across horizons (averaged over {num_runs} runs)"
            )
        )
        
        # Also print LipBR last-arm pseudo-regret summary across horizons (averaged).
        print(
            "\n"
            + lipbr_last_arm_regret_table_markdown(
                histories_by_policy_avg, decimals=1,
                title=f"LipBR pseudo-regret vs last arm (final cumulative, averaged over {num_runs} runs)"
            )
        )
        
        # Print average pseudo-regret vs LipBR last arm for all policies (averaged).
        print(
            "\n"
            + avg_regret_vs_lipbr_last_arm_table_markdown(
                histories_by_policy_avg, decimals=3,
                title=f"Average pseudo-regret vs LipBR last arm (averaged over {num_runs} runs)"
            )
        )
        
        # ---------------------------------------------------------------------
        # Saving averaged results (largest horizon only)
        # ---------------------------------------------------------------------
        if 'save_history' in globals():
            T_max = max(T_list)
            meta_common = {
                "n": str(n),
                "T": str(T_max),
                "demand_mu": ",".join(map(lambda x: f"{x:.3f}", demand_mu)),
                "num_runs": str(num_runs),
                "shared_randomness": "True",
            }
            hist_lipbr_Tmax = histories_by_policy_avg["LipBR"].get(T_max, {})
            hist_norepo_Tmax = histories_by_policy_avg["NoRepo"].get(T_max, {})
            hist_uniform_Tmax = histories_by_policy_avg["Uniform"].get(T_max, {})
            
            grid_m_Tmax = recommended_grid_m(
                T=T_max, n=n, cost_reposition=COST_REPO, cost_lost=COST_LOST, max_arms=5000
            )
            
            lipbr_meta = {
                **meta_common,
                "policy": "LipBR",
                "grid_m": str(grid_m_Tmax),
                "H": str(5.0),
                "cost_reposition": str(COST_REPO),
                "cost_lost": str(COST_LOST),
                "seed": "averaged",
            }
            norepo_meta = {
                **meta_common,
                "policy": "NoRepo",
                "cost_reposition": str(COST_REPO),
                "cost_lost": str(COST_LOST),
                "seed": "averaged",
            }
            uniform_meta = {
                **meta_common,
                "policy": "Uniform",
                "cost_reposition": str(COST_REPO),
                "cost_lost": str(COST_LOST),
                "seed": "averaged",
            }
            params_md = {
                "n": n,
                "T": T_max,
                "m (grid_m)": grid_m_Tmax,
                "H": 5.0,
                "demand_mu": ",".join(map(lambda x: f"{x:.3f}", demand_mu)),
                "cost_reposition (LipBR)": COST_REPO,
                "cost_lost (LipBR)": COST_LOST,
                "cost_reposition (NoRepo)": COST_REPO,
                "cost_lost (NoRepo)": COST_LOST,
                "cost_reposition (Uniform)": COST_REPO,
                "cost_lost (Uniform)": COST_LOST,
                "num_runs": num_runs,
                "shared_randomness": True,
            }
            lipbr_csv = save_history(hist_lipbr_Tmax, label=f"LipBR_n{n}", outdir="numerical_results", extra_meta=lipbr_meta, write_metadata_md=True, metadata_md_params=params_md)
            norepo_csv = save_history(hist_norepo_Tmax, label=f"NoRepo_n{n}", outdir="numerical_results", extra_meta=norepo_meta, write_metadata_md=False, metadata_md_params=params_md)
            uniform_csv = save_history(hist_uniform_Tmax, label=f"Uniform_n{n}", outdir="numerical_results", extra_meta=uniform_meta, write_metadata_md=False, metadata_md_params=params_md)
            print(f"Saved: {lipbr_csv}")
            print(f"Saved: {norepo_csv}")
            print(f"Saved: {uniform_csv}")
        else:
            print("Skipping file saves: save_history not loaded in kernel.")


===== Running experiments for n=2 =====

[ n=2 ] **Average total cost across horizons (averaged over 20 runs)**

| Policy | T=1000 | T=2000 | T=3000 |
|:-------|:----------:|:----------:|:----------:|
| LipBR | 6.322 | 6.363 | 6.409 |
| NoRepo | 7.230 | 7.289 | 7.306 |
| Uniform | 6.473 | 6.487 | 6.506 |

**LipBR pseudo-regret vs last arm (final cumulative, averaged over 20 runs)**

| Metric | T=1000 | T=2000 | T=3000 |
|:-------|:----------:|:----------:|:----------:|
| Final cum. pseudo-regret | 712.2 | -3258.2 | -3169.5 |

**Average pseudo-regret vs LipBR last arm (averaged over 20 runs)**

| Policy | T=1000 | T=2000 | T=3000 |
|:-------|:----------:|:----------:|:----------:|
| LipBR | 0.712 | -1.629 | -1.056 |
| NoRepo | 1.554 | -0.725 | -0.142 |
| Uniform | 0.797 | -1.527 | -0.942 |
Saved: /Users/hanshengjiang/Documents/GitHubLocal/LipschitzMDP/numerical_results/20251129-154057/lipbr_n2.csv
Saved: /Users/hanshengjiang/Documents/GitHubLocal/LipschitzMDP/numerical_results/20251129